In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import logging
from gsheet import *
from mailer import *
from qr_code import *
from payment_data import *


"""
If logging in results in an error, delete the 'token.json' to re-authenticate.
"""

"\nIf logging in results in an error, delete the 'token.json' to re-authenticate.\n"

In [21]:
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    level=logging.INFO
)

In [22]:
sheet = Sheets(SPREADSHEET_ID)

[INFO] file_cache is only supported with oauth2client<4.0.0


In [23]:
# sheet_names = sheet.get_sheet_names()
# sheet_name = '4pvs_all'
# sheet_name = 'PLATBY_PODZIM_2022_filtered'
# sheet_name="ExportNoFormulas"
# sheet_name="SkautisExport20231107"
# sheet_name="PlatbyJaro2025-racDodatek"
sheet_name="PlatbyPodzim2025"
# sheet_name = 'TESTv2'
if 'TEST' in sheet_name:
    testmode=True
else:
    testmode=False
df = sheet.get_dataframe(sheet_name)

In [24]:
df

,Jméno,Příjmení,Osoba,Přezdívka,Evidenční číslo,Jednotka,Typ jednotky,Typ členství,Kategorie,Odebírané časopisy,...,Mobil (další),Telefon (další),Telefon / mobil (hlavní),Telefon do práce,Otec: mail,Otec: telefon,Matka: mail,Matka: telefon,Ostatní: mail,Ostatní: telefon
0,Radka,Vachatová,Vachatová Radka,Redy,113.04,4. přístav Jana Nerudy Praha,Středisko,Řádné,Ostatní,Skautský svět,...,733705515,,731533258,None,None,None,None,None,None,None
1,Hanka,Svrčková,Svrčková Hanka,,113.04,4. přístav Jana Nerudy Praha,Středisko,Řádné,Ostatní,Skautský svět,...,None,None,None,None,None,None,None,None,None,None
2,Karel,Dvořák,Dvořák Karel,Šárlí,113.04.004-1,Albatrosové,Oddíl,Řádné,Ostatní,Skautský svět,...,,,721978545,None,None,None,None,None,None,None
3,David,Svoboda,Svoboda David,Cedník,113.04.004-1,Albatrosové,Oddíl,Řádné,Ostatní,"Skautský svět, Skauting",...,,,732131483,None,None,None,None,None,None,None
4,Jiří,Svrček,Svrček Jiří,Jikra,113.04.004-1,Albatrosové,Oddíl,Řádné,Rover,Skautský svět,...,,731893783,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Lada,Zámečníková,Zámečníková Lada,,113.04.004-4,Želvy,Oddíl,Řádné,Světluška,BenJáMína,...,,,721000730,,martinz6492@seznam.cz,737859247,veronika.zamecnikova@omska.cz,724028382,None,None
246,Mariana,Novotná,Novotná Mariana,,113.04.004-4,Želvy,Oddíl,Řádné,Světluška,BenJáMína,...,,,,,marian@natur.cuni.cz,604649422,lenka.novotna.81@seznam.cz,607667772,None,None
247,Klára,Mašková,Mašková Klára,,113.04.004-4,Želvy,Oddíl,Řádné,Světluška,BenJáMína,...,,,,,masekj86@gmail.com,737842143,daniela.maskova.cz@gmail.com,602336463,None,None
248,Thea,Petrová,Petrová Thea,,113.04.004-4,Želvy,Oddíl,Řádné,Světluška,Světýlko,...,,,,,jakub.peter@gmail.com,604822309,daska.v@gmail.com,605444881,None,None


In [25]:
NEEDED_COLS = NeededColumns()
pdf=PaymentsDataFrame(df, NEEDED_COLS)

u:\Users\Honza\scth\projects\4pvs_prispevky_qr_kod\payment_data.py:112: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  payments = df.loc[:, cols.czk_amounts].fillna(0)
[INFO] Of 250 people, missing email: 1,
	emailable paid: 21
	emailable unpaid: 228.
 Also there were 0 invalid emails.


In [26]:
pdf.df_missing_email

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,Poplatek,Zaplaceno,valid_addresses,invalid_addresses
1,Svrčková Hanka,4. přístav Jana Nerudy Praha,None,None,None,None,None,None,1000,None,(),()


In [27]:
emails = []

"""
!!!!!WARNING! MAY BE CURRENTLY SET TO EMAIL ALL AVAILABLE EMAIL ADDRESSES!!!!!

Modify this setting on second line:
    recepient_addresses = row.valid_addresses
    
Modify order in in class NeededColumns
"""

for i, row in pdf.df_emailable_unpaid.iterrows():
    recepient_addresses = row.valid_addresses[:2]
    pi = PaymentInfo.from_df_row(row, NEEDED_COLS)
    email = PaymentEmail(pi, recepient_addresses, testmode=testmode)
    emails.append(email.msg)

In [28]:
pdf.df_emailable_unpaid

,Osoba,Jednotka,Registrační číslo,E-mail (hlavní),Otec: mail,Matka: mail,E-mail (další),Ostatní: mail,Poplatek,Zaplaceno,valid_addresses,invalid_addresses
0,Vachatová Radka,4. přístav Jana Nerudy Praha,507,radka.vachatova@seznam.cz,None,None,None,None,1000,0,"(""E-mail (hlavní)"" <radka.vachatova@seznam.cz>,)",()
3,Svoboda David,Albatrosové,10010,cednik@email.cz,None,None,None,None,1000,0,"(""E-mail (hlavní)"" <cednik@email.cz>,)",()
4,Svrček Jiří,Albatrosové,10013,None,None,None,tajfun.svrcek@seznam.cz,None,2000,0,"(""E-mail (další)"" <tajfun.svrcek@seznam.cz>,)",()
5,Mrkos Jan,Albatrosové,10015,hondzik@skaut.cz,None,None,"hondzik@skautka.cz, jan.mrkos@skaut.cz, mrkos....",None,1000,0,"(""E-mail (hlavní)"" <hondzik@skaut.cz>, ""E-mail...",()
6,Milec Oldřich,Albatrosové,10017,oldamilec@seznam.cz,None,None,olda.m@skaut.cz,None,1000,0,"(""E-mail (hlavní)"" <oldamilec@seznam.cz>, ""E-m...",()
...,...,...,...,...,...,...,...,...,...,...,...,...
244,Socháňová Agáta,Želvy,60004,None,petr.sochan@gmail.com,ada.tzoneva@gmail.com,None,None,1000,0,"(""Matka: mail"" <ada.tzoneva@gmail.com>, ""Otec:...",()
245,Zámečníková Lada,Želvy,60006,None,martinz6492@seznam.cz,veronika.zamecnikova@omska.cz,None,None,2000,0,"(""Matka: mail"" <veronika.zamecnikova@omska.cz>...",()
246,Novotná Mariana,Želvy,60011,None,marian@natur.cuni.cz,lenka.novotna.81@seznam.cz,None,None,1000,0,"(""Matka: mail"" <lenka.novotna.81@seznam.cz>, ""...",()
247,Mašková Klára,Želvy,60018,None,masekj86@gmail.com,daniela.maskova.cz@gmail.com,None,None,1000,0,"(""Matka: mail"" <daniela.maskova.cz@gmail.com>,...",()


In [33]:
with open(f'./test.eml', 'wb') as f:
    f.write(bytes(emails[35]))

## ⚡ Danger Zone ⚡

In [30]:
mailer = Mailer()
sent = []

[INFO] file_cache is only supported with oauth2client<4.0.0


In [31]:
for i, email in enumerate(emails[:]):
    print(f'Sending email {i+1}/{len(emails)}:')
    sent_msg = mailer.send_message(email)
    sent.append((email, sent_msg))

Sending email 1/228:


[INFO] Message Id: 199dfb81ea705094 sent.


Sending email 2/228:


[INFO] Message Id: 199dfb82163bd264 sent.


Sending email 3/228:


...

[INFO] Message Id: 199dfbb533bc6cf4 sent.


Sending email 227/228:


[INFO] Message Id: 199dfbb5956d337f sent.


Sending email 228/228:


[INFO] Message Id: 199dfbb5d0b2d00d sent.
